In [13]:
import pickle
import sys
import tensorflow as tf
from tqdm import tqdm

In [14]:
def get_labels():
    with open("/Users/pkaur/tensorflow/SignLanguage/InceptionModel/retrained_labels.txt", 'r') as fin:
        labels = [line.rstrip('\n') for line in fin]
    return labels

In [15]:
def predict_on_frames(frames):
    """Given a list of frames, predict all their classes."""
    # Unpersists graph from file
    with tf.gfile.FastGFile("/Users/pkaur/tensorflow/SignLanguage/InceptionModel/retrained_graph.pb", 'rb') as fin:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(fin.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        frame_predictions = []
        # There are currently 4 Test Files, testing the 4th one here
        image_path = '/Users/pkaur/tensorflow/SignLanguage/SignDataTest/TestSet3/'
        pbar = tqdm(total=len(frames))
        for i, frame in enumerate(frames):
            filename = frame[0]
            label = frame[1]

            # Get the image path.
            image = image_path + label +"/"+ filename

            # Read in the image_data
            image_data = tf.gfile.FastGFile(image, 'rb').read()

            try:
                predictions = sess.run(
                    softmax_tensor, {'DecodeJpeg/contents:0': image_data}
                )
                prediction = predictions[0]
            except KeyboardInterrupt:
                print("You quit with ctrl+c")
                sys.exit()
            except:
                print("Error making prediction, continuing.")
                continue

            # Save the probability that it's each of our classes.
            frame_predictions.append([prediction, label])

            if i > 0 and i % 10 == 0:
                pbar.update(10)

        pbar.close()

        return frame_predictions

In [16]:
def get_accuracy(predictions, labels):
    """After predicting on each batch, check that batch's
    accuracy to make sure things are good to go. This is
    a simple accuracy metric, and so doesn't take confidence
    into account, which would be a better metric to use to
    compare changes in the model."""
    correct = 0
    for frame in predictions:
        # Get the highest confidence class.
        this_prediction = frame[0].tolist()
        this_label = frame[1]

        max_value = max(this_prediction)
        max_index = this_prediction.index(max_value)
        predicted_label = labels[max_index]
        print this_label +" -> "+predicted_label

        # Now see if it matches.
        if predicted_label == this_label:
            correct = correct + 1

    print("Correctly Labelled Frames : %d" % correct)
    print("Total Frames : %d" % len(predictions))
    accuracy = float(float(correct) / float(len(predictions)))
    return accuracy

In [17]:
def main():
        labels = get_labels()
        with open('/Users/pkaur/tensorflow/SignLanguage/test-data-labeled-images.pkl', 'rb') as fin:
            frames = pickle.load(fin)

        # Predict on this batch and get the accuracy.
        predictions = predict_on_frames(frames)
        
        accuracy = get_accuracy(predictions, labels)
        print("Batch accuracy: %.5f" % accuracy)

        # Save it.
        with open('/Users/pkaur/tensorflow/SignLanguage/test-data-predicted-frames.pkl', 'wb') as fout:
            pickle.dump(predictions, fout)

        print("Done.")

In [18]:
if __name__ == '__main__':
    main()

 96%|█████████▌| 220/229 [02:12<00:04,  1.98it/s]


again -> again
again -> cool
again -> again
again -> again
again -> again
again -> again
again -> again
again -> again
again -> again
again -> again
again -> your
again -> your
again -> again
again -> fingerspell
again -> again
again -> again
again -> again
again -> again
cool -> cool
cool -> fingerspell
cool -> your
cool -> cool
cool -> your
cool -> cool
cool -> your
cool -> cool
cool -> cool
cool -> cool
cool -> have to
cool -> fingerspell
cool -> cool
cool -> fingerspell
cool -> cool
cool -> your
cool -> your
cool -> have to
cool -> your
cool -> have to
cool -> your
dont know -> why
dont know -> why
dont know -> fingerspell
dont know -> fingerspell
dont know -> fingerspell
dont know -> fingerspell
dont know -> fingerspell
dont know -> why
dont know -> fingerspell
dont know -> why
dont know -> fingerspell
dont know -> fingerspell
dont know -> why
dont know -> fingerspell
dont know -> fingerspell
dont know -> fingerspell
dont know -> why
dont know -> their
dont know -> why
dont know -